In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
%load_ext dotenv
%dotenv

cannot find .env file


In [32]:
from langchain.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("/Users/lavesh/Downloads/Invoices 2021")

In [34]:
raw_documents = loader.load()

Object 4 0 found
Object 2 0 found


In [35]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [36]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap  = 50
)

In [37]:
documents = text_splitter.split_documents(raw_documents)

In [38]:
from langchain.embeddings import SentenceTransformerEmbeddings

embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [39]:
from langchain.vectorstores import Chroma

db = Chroma.from_documents(documents, embeddings)

In [40]:
retriever = db.as_retriever(
    search_type="mmr",  # Also test "similarity"
    search_kwargs={"k": 8},
)

In [57]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

llm = ChatOpenAI(model_name="gpt-3.5-turbo")
memory = ConversationSummaryMemory(
    llm=llm, memory_key="chat_history", return_messages=True
)
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

In [58]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [59]:
questions = [
    # "Which city is KCA Event Management In?",
    # "What is the price of PCR Test?",
    # "What are the bank details of High Caliber?"
    # "What is the hourly cost for a promotion?",
    # "What is the transportation cost to Yas Island?",
    "Where is SYSCOM located in?"
]

for question in questions:
    result = qa(question)
    print(f"-> *Question*: {question} \n")
    print(f"*Answer*: {result['answer']} \n")

-> *Question*: Where is SYSCOM located in? 

*Answer*: The location of SYSCOM is not provided in the given context. 

